# TODO


- Replace of NULL values in the csv file by 0 (for class-based metrics only)
- Add check of NULL values at the end (Pandas) to be sure there are not anymore in the dataset
- Add the following metrics:
    1. Density of built floor area (sum of built pixels' height / area of block)

<center> <font size=5> <h1>Define working environment</h1> </font> </center> 

The following cells are used to: 
- Import needed libraries
- Set the environment variables for Python, Anaconda, GRASS GIS and R statistical computing 
- Define the ["GRASSDATA" folder](https://grass.osgeo.org/grass73/manuals/helptext.html), the name of "location" and "mapset" where you will to work.

**Import libraries**

In [186]:
## Import libraries needed for setting parameters of operating system 
import os
import sys

<center> <font size=3> <h3>Environment variables when working on Linux Mint</h3> </font> </center> 

**Set 'Python' and 'GRASS GIS' environment variables**

Here, we set [the environment variables allowing to use of GRASS GIS](https://grass.osgeo.org/grass64/manuals/variables.html) inside this Jupyter notebook. Please change the directory path according to your own system configuration.

In [187]:
### Define GRASS GIS environment variables for LINUX UBUNTU Mint 18.1 (Serena)
# Check is environmental variables exists and create them (empty) if not exists.
if not 'PYTHONPATH' in os.environ:
    os.environ['PYTHONPATH']=''
if not 'LD_LIBRARY_PATH' in os.environ:
    os.environ['LD_LIBRARY_PATH']=''
# Set environmental variables
os.environ['GISBASE'] = '/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu'
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'bin')
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'script')
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'lib')
#os.environ['PATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'etc','python')
os.environ['PYTHONPATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'etc','python')
os.environ['PYTHONPATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'etc','python','grass')
os.environ['PYTHONPATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'etc','python','grass','script')
os.environ['PYTHONLIB'] = '/usr/lib/python2.7'
os.environ['LD_LIBRARY_PATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'lib')
os.environ['GIS_LOCK'] = '$$'
os.environ['GISRC'] = os.path.join(os.environ['HOME'],'.grass7','rc')
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['HOME'],'.grass7','addons')
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['HOME'],'.grass7','addons','bin')
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['HOME'],'.grass7','addons')
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['HOME'],'.grass7','addons','scripts')

## Define GRASS-Python environment
sys.path.append(os.path.join(os.environ['GISBASE'],'etc','python'))

**Import GRASS Python packages**

In [188]:
## Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup

## Import libraries needed to call GRASS using Python
import grass.script as gscript
from grass.script import core as grass

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

**Display current environment variables of your computer**

In [189]:
## Display the current defined environment variables
for key in os.environ.keys():
    print "%s = %s \t" % (key,os.environ[key])

MDMSESSION = mate 	
MANDATORY_PATH = /usr/share/gconf/mate.mandatory.path 	
MATE_DESKTOP_SESSION_ID = this-is-deprecated 	
LESSOPEN = | /usr/bin/lesspipe %s 	
MDM_LANG = fr_BE.UTF-8 	
LOGNAME = tais 	
USER = tais 	
HOME = /home/tais 	
XDG_VTNR = 9 	
PATH = /usr/local/bin:/home/tais/BIN:/home/tais/bin:/home/tais/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/bin:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/script:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/lib:/home/tais/.grass7/addons:/home/tais/.grass7/addons/bin:/home/tais/.grass7/addons:/home/tais/.grass7/addons/scripts:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/bin:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/scripts:/home/tais/.grass7/addons/scripts:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/bin:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

<center> <font size=5> <h1>Define functions</h1> </font> </center> 

This section of the notebook is dedicated to defining functions which will then be called later in the script. If you want to create your own functions, define them here.

### Function for computing processing time

The "print_processing_time" is used to calculate and display the processing time for various stages of the processing chain. At the beginning of each major step, the current time is stored in a new variable, using [time.time() function](https://docs.python.org/2/library/time.html). At the end of the stage in question, the "print_processing_time" function is called and takes as argument the name of this new variable containing the recorded time at the beginning of the stage, and an output message.

In [190]:
## Import library for managing time in python
import time  

## Function "print_processing_time()" compute processing time and printing it.
# The argument "begintime" wait for a variable containing the begintime (result of time.time()) of the process for which to compute processing time.
# The argument "printmessage" wait for a string format with information about the process. 
def print_processing_time(begintime, printmessage):    
    endtime=time.time()           
    processtime=endtime-begintime
    remainingtime=processtime

    days=int((remainingtime)/86400)
    remainingtime-=(days*86400)
    hours=int((remainingtime)/3600)
    remainingtime-=(hours*3600)
    minutes=int((remainingtime)/60)
    remainingtime-=(minutes*60)
    seconds=round((remainingtime)%60,1)

    if processtime<60:
        finalprintmessage=str(printmessage)+str(seconds)+" seconds"
    elif processtime<3600:
        finalprintmessage=str(printmessage)+str(minutes)+" minutes and "+str(seconds)+" seconds"
    elif processtime<86400:
        finalprintmessage=str(printmessage)+str(hours)+" hours and "+str(minutes)+" minutes and "+str(seconds)+" seconds"
    elif processtime>=86400:
        finalprintmessage=str(printmessage)+str(days)+" days, "+str(hours)+" hours and "+str(minutes)+" minutes and "+str(seconds)+" seconds"
    
    return finalprintmessage

### Function for creation of configuration file for r.li (landscape units provided as polygons) (multiprocessed)

In [191]:
##### Function that create the r.li configuration file for a list of landcover raster.
### It enable to create in one function as many configuration file as the number of raster provided in 'listoflandcoverraster'.
### It could be use only in case study with a several landcover raster and only one landscape unit layer.
### So, the landscape unit layer if fixed and there are the landcover raster which change. 
# 'listoflandcoverraster' wait for a list with the name (string) of landcover rasters.
# 'landscape_polygons' wait for the name (string) of the vector layer containing the polygons to be used as landscape units.
# 'masklayerhardcopy' wait for a boolean value (True/False) depending if the user want to create hard copy of the landscape units mask layers or not.
# 'returnlistpath' wait for a boolean value (True/False) according to the fact that a list containing the path to the configuration files is desired.
# 'ncores' wait for a integer corresponding to the number of desired cores to be used for parallelization.

# Import libraries for multiprocessing 
import multiprocessing
from multiprocessing import Pool
from functools import partial 

# Function that copy the landscape unit raster masks on a new layer with name corresponding to the current 'landcover_raster'
def copy_landscapeunitmasks(current_landcover_raster,base_landcover_raster,landscape_polygons,landscapeunit_bbox,cat):
    ### Copy the landscape units mask for the current 'cat'
    # Define the name of the current "current_landscapeunit_rast" layer
    current_landscapeunit_rast=current_landcover_raster.split("@")[0]+"_"+landscape_polygons.split("@")[0]+"_"+str(cat)          
    base_landscapeunit_rast=base_landcover_raster.split("@")[0]+"_"+landscape_polygons.split("@")[0]+"_"+str(cat)          
    # Copy the the landscape unit created for the first landcover map in order to match the name of the current landcover map
    gscript.run_command('g.copy', overwrite=True, quiet=True, raster=(base_landscapeunit_rast,current_landscapeunit_rast))
    # Add the line to the text variable
    text="MASKEDOVERLAYAREA "+current_landscapeunit_rast+"|"+landscapeunit_bbox[cat]
    return text

# Function that create the r.li configuration file for the base landcover raster and then for all the binary rasters
def create_rli_configfile(listoflandcoverraster,landscape_polygons,
                          masklayerhardcopy=False,returnlistpath=True,ncores=2):
    # Check if 'listoflandcoverraster' is not empty
    if len(listoflandcoverraster)==0:
        sys.exit("The list of landcover raster is empty and should contain at least one raster name")
    # Check if rasters provided in 'listoflandcoverraster' exists to avoid error in mutliprocessing 
    for cur_rast in listoflandcoverraster:
        try:
            mpset=cur_rast.split("@")[1]
        except:
            mpset=""
        if cur_rast.split("@")[0] not in [x[0] for x in gscript.list_pairs(type='raster',mapset=mpset)]:
            sys.exit('Raster <%s> not found' %cur_rast)
    # Check if rasters provided in 'listoflandcoverraster' have the same extend and spatial resolution 
    raster={}
    for x, rast in enumerate(raster_list):
        raster[x]=gscript.raster_info(rast)
    key_list=raster.keys()
    for x in key_list[1:]:
        for info in ('north','south','east','west','ewres','nsres'):
            if not raster[0][info]==raster[x][info]:
                sys.exit("Some raster provided in the list have different spatial resolution or extend, please check")    
    # Get the version of GRASS GIS 
    version=grass.version()['version'].split('.')[0]
    # Define the folder to save the r.li configuration files
    if sys.platform=="win32":
        rli_dir=os.path.join(os.environ['APPDATA'],"GRASS"+version,"r.li")
    else: 
        rli_dir=os.path.join(os.environ['HOME'],".grass"+version,"r.li")
    if not os.path.exists(rli_dir):
        os.makedirs(rli_dir)
    ## Create an ordered list with the 'cat' value of landscape units to be processed.
    list_cat=[int(x) for x in gscript.parse_command('v.db.select', quiet=True, 
                                                        map=landscape_polygons, column='cat', flags='c')]
    list_cat.sort()
    # Declare a empty dictionnary which will contains the north, south, east, west values for each landscape unit
    landscapeunit_bbox={}
    # Declare a empty list which will contain the path of the configation files created
    listpath=[]
    # Declare a empty string variable which will contains the core part of the r.li configuration file
    maskedoverlayarea_1=""
    # Duplicate 'listoflandcoverraster' in a new variable called 'tmp_list'
    tmp_list=list(listoflandcoverraster)
    # Set the current landcover raster as the first of the list
    base_landcover_raster=tmp_list.pop(0) #The pop function return the first item of the list and delete it from the list at the same time
    # Loop trough the landscape units
    for cat in list_cat:
        # Extract the current landscape unit polygon as temporary vector
        tmp_vect="tmp_"+base_landcover_raster.split("@")[0]+"_"+landscape_polygons.split("@")[0]+"_"+str(cat)
        gscript.run_command('v.extract', overwrite=True, quiet=True, 
                            input=landscape_polygons, cats=cat, output=tmp_vect)
        # Set region to match the extent of the current landscape polygon, with resolution and alignement matching the landcover raster
        gscript.run_command('g.region', vector=tmp_vect, align=base_landcover_raster)
        # Rasterize the landscape unit polygon
        landscapeunit_rast=tmp_vect[4:]
        gscript.run_command('v.to.rast', overwrite=True, quiet=True, input=tmp_vect, output=landscapeunit_rast, use='cat', memory='3000')
        # Remove temporary vector
        gscript.run_command('g.remove', quiet=True, flags="f", type='vector', name=tmp_vect)
        # Set the region to match the raster landscape unit extent and save the region info in a dictionary
        region_info=gscript.parse_command('g.region', raster=landscapeunit_rast, flags='g')
        n=str(round(float(region_info['n']),5)) #the config file need 5 decimal for north and south
        s=str(round(float(region_info['s']),5))
        e=str(round(float(region_info['e']),6)) #the config file need 6 decimal for east and west
        w=str(round(float(region_info['w']),6))
        # Save the coordinates of the bbox in the dictionary (n,s,e,w)
        landscapeunit_bbox[cat]=n+"|"+s+"|"+e+"|"+w
        # Add the line to the maskedoverlayarea_1 variable
        maskedoverlayarea_1+="MASKEDOVERLAYAREA "+landscapeunit_rast+"|"+landscapeunit_bbox[cat]+"\n"

    # Compile the content of the r.li configuration file
    config_file_content="SAMPLINGFRAME 0|0|1|1\n"
    config_file_content+=maskedoverlayarea_1
    config_file_content+="RASTERMAP "+base_landcover_raster+"\n"
    config_file_content+="VECTORMAP "+landscape_polygons+"\n"

    # Create a new file and save the content
    configfilename=base_landcover_raster.split("@")[0]+"_"+landscape_polygons.split("@")[0]
    path=os.path.join(rli_dir,configfilename)
    listpath.append(path)
    f=open(path, 'w')
    f.write(config_file_content)
    f.close()
    
    # Continue creation of r.li configuration file and landscape unit raster the rest of the landcover raster provided
    while len(tmp_list)>0:
        # Initialize 'maskedoverlayarea_2' variable as an empty string
        maskedoverlayarea_2=""
        # Set the current landcover raster as the first of the list
        current_landcover_raster=tmp_list.pop(0) #The pop function return the first item of the list and delete it from the list at the same time
        if masklayerhardcopy: # If the user asked for hard copy of the landscape units mask layers
            # Copy all the landscape units masks for the current landcover raster
            p=Pool(ncores) #Create a pool of processes and launch them using 'map' function
            func=partial(copy_landscapeunitmasks,current_landcover_raster,base_landcover_raster,landscape_polygons,landscapeunit_bbox) # Set fixed argument of the function
            maskedoverlayarea_2=p.map(func,list_cat) # Launch the processes for as many items in the list and get the ordered results using map function
            p.close()
            p.join()
            # Compile the content of the r.li configuration file
            config_file_content="SAMPLINGFRAME 0|0|1|1\n"
            config_file_content+="\n".join(maskedoverlayarea_2)+"\n"
            config_file_content+="RASTERMAP "+current_landcover_raster+"\n"
            config_file_content+="VECTORMAP "+landscape_polygons+"\n"
        else: # If the user not asked for hard copy
            # Compile the content of the r.li configuration file
            config_file_content="SAMPLINGFRAME 0|0|1|1\n"
            config_file_content+=maskedoverlayarea_1  # If user do not asked for hard copy, the mask layers are the same than for the first configuration file
            config_file_content+="RASTERMAP "+current_landcover_raster+"\n"  # But the name of the RASTERMAP should be the one of the current landcover raster
            config_file_content+="VECTORMAP "+landscape_polygons+"\n"
        # Create a new file and save the content
        configfilename=current_landcover_raster.split("@")[0]+"_"+landscape_polygons.split("@")[0]
        path=os.path.join(rli_dir,configfilename)
        listpath.append(path)
        f=open(path, 'w')
        f.write(config_file_content)
        f.close()
    
    # Return a list of path of configuration files creates if option actived
    if returnlistpath:
        return list_cat,listpath
    else:
        return list_cat

### Function for creation of binary raster from a categorical raster (multiprocessed)

In [192]:
###### Function creating a binary raster for each category of a base raster. 
### The function run within the current region. If a category do not exists in the current region, no binary map will be produce
# 'categorical_raster' wait for the name of the base raster to be used. It is the one from which one binary raster will be produced for each category value
# 'prefix' wait for a string corresponding to the prefix of the name of the binary raster which will be produced
# 'setnull' wait for a boolean value (True, False) according to the fact that the output binary should be 1/0 or 1/null
# 'returnlistraster' wait for a boolean value (True, False) regarding to the fact that a list containing the name of binary raster is desired as return of the function
# 'category_list' wait for a list of interger corresponding to specific category of the base raster to be used 
# 'ncores' wait for a integer corresponding to the number of desired cores to be used for parallelization

# Import libraries for multiprocessing 
import multiprocessing
from multiprocessing import Pool
from functools import partial   

def create_binary_raster(categorical_raster,prefix="binary",setnull=False,returnlistraster=True,category_list=None,ncores=2):
    # Check if raster exists to avoid error in mutliprocessing 
    try:
        mpset=categorical_raster.split("@")[1]
    except:
        mpset=""
    if categorical_raster not in gscript.list_strings(type='raster',mapset=mpset):
        sys.exit('Raster <%s> not found' %categorical_raster)
    # Check for number of cores doesnt exceed available
    nbcpu=multiprocessing.cpu_count()
    if ncores>=nbcpu:
        ncores=nbcpu-1
    returnlist=[] #Declare empty list for return
    #gscript.run_command('g.region', raster=categorical_raster, quiet=True) #Set the region
    null='null()' if setnull else '0' #Set the value for r.mapcalc
    minclass=1 if setnull else 2 #Set the value to check if the binary raster is empty
    if category_list == None: #If no category_list provided
        category_list=[cl for cl in gscript.parse_command('r.category',map=categorical_raster,quiet=True)]
    for i,x in enumerate(category_list):  #Make sure the format is UTF8 and not Unicode
        category_list[i]=x.encode('UTF8')
    category_list.sort(key=float) #Sort the raster categories in ascending.
    p=Pool(ncores) #Create a pool of processes and launch them using 'map' function
    func=partial(get_binary,categorical_raster,prefix,null,minclass) # Set the two fixed argument of the function
    returnlist=p.map(func,category_list) # Launch the processes for as many items in the 'functions_name' list and get the ordered results using map function
    p.close()
    p.join()
    if returnlistraster:
        return returnlist

#### Function that extract binary raster for a specified class (called in 'create_binary_raster' function)
def get_binary(categorical_raster,prefix,null,minclass,cl):
    binary_class=prefix+"_"+cl
    gscript.run_command('r.mapcalc', expression=binary_class+'=if('+categorical_raster+'=='+str(cl)+',1,'+null+')',overwrite=True, quiet=True)
    if len(gscript.parse_command('r.category',map=binary_class,quiet=True))>=minclass:  #Check if created binary is not empty
        return binary_class
    else:
        gscript.run_command('g.remove', quiet=True, flags="f", type='raster', name=binary_class)

### Function for computation of spatial metrics at landscape level (multiprocessed)

In [193]:
##### Function that compute different landscape metrics (spatial metrics) at landscape level. 
### The metric computed are "dominance","pielou","renyi","richness","shannon","simpson".
### It is important to set the computation region before runing this script so that it match the extent of the 'raster' layer.
# 'configfile' wait for the path (string) to the configuration file corresponding to the 'raster' layer.
# 'raster' wait for the name (string) of the landcover map on which landscape metrics will be computed.
# 'returnlistresult' wait for a boolean value (True/False) according to the fact that a list containing the path to the result files is desired.
# 'ncores' wait for a integer corresponding to the number of desired cores to be used for parallelization.

# Import libraries for multiprocessing 
import multiprocessing
from multiprocessing import Pool
from functools import partial   

def compute_landscapelevel_metrics(configfile, raster, spatial_metric):
    filename=raster.split("@")[0]+"_%s" %spatial_metric
    outputfile=os.path.join(os.path.split(configfile)[0],"output",filename)
    if spatial_metric=='renyi': # The alpha parameter was set to 2 as in https://en.wikipedia.org/wiki/R%C3%A9nyi_entropy
        gscript.run_command('r.li.%s' %spatial_metric, overwrite=True,
                            input=raster,config=configfile,alpha='2', output=filename)
    else:
        gscript.run_command('r.li.%s' %spatial_metric, overwrite=True,
                    input=raster,config=configfile, output=filename)
    return outputfile
    
def get_landscapelevel_metrics(configfile, raster, returnlistresult=True, ncores=2):
    # Check if raster exists to avoid error in mutliprocessing 
    try:
        mpset=raster.split("@")[1]
    except:
        mpset=""
    if raster not in gscript.list_strings(type='raster',mapset=mpset):
        sys.exit('Raster <%s> not found' %raster)
    # Check if configfile exists to avoid error in mutliprocessing 
    if not os.path.exists(configfile):
        sys.exit('Configuration file <%s> not found' %configfile)
    # List of metrics to be computed
    spatial_metric_list=["dominance","pielou","renyi","richness","shannon","simpson"]
    # Check for number of cores doesnt exceed available
    nbcpu=multiprocessing.cpu_count()
    if ncores>=nbcpu:
        ncores=nbcpu-1
        if ncores>len(spatial_metric_list):
            ncores=len(spatial_metric_list)  #Adapt number of cores to number of metrics to compute
    #Declare empty list for return
    returnlist=[] 
    # Create a new pool
    p=Pool(ncores)
    # Set the two fixed argument of the 'compute_landscapelevel_metrics' function
    func=partial(compute_landscapelevel_metrics,configfile, raster)
    # Launch the processes for as many items in the 'functions_name' list and get the ordered results using map function
    returnlist=p.map(func,spatial_metric_list)
    p.close()
    p.join()
    # Return list of paths to result files
    if returnlistresult:
        return returnlist

### Function for computation of spatial metrics at class level (multiprocessed)

In [194]:
##### Function that compute different landscape metrics (spatial metrics) at class level. 
### The metric computed are "patch number (patchnum)","patch density (patchdensity)","mean patch size(mps)",
### "coefficient of variation of patch area (padcv)","range of patch area size (padrange)",
### "standard deviation of patch area (padsd)", "shape index (shape)", "edge density (edgedensity)".
### It is important to set the computation region before runing this script so that it match the extent of the 'raster' layer.
# 'configfile' wait for the path (string) to the configuration file corresponding to the 'raster' layer.
# 'raster' wait for the name (string) of the landcover map on which landscape metrics will be computed.
# 'returnlistresult' wait for a boolean value (True/False) according to the fact that a list containing the path to the result files is desired.
# 'ncores' wait for a integer corresponding to the number of desired cores to be used for parallelization.

# Import libraries for multiprocessing 
import multiprocessing
from multiprocessing import Pool
from functools import partial   

def compute_classlevel_metrics(configfile, raster, spatial_metric):
    filename=raster.split("@")[0]+"_%s" %spatial_metric
    gscript.run_command('r.li.%s' %spatial_metric, overwrite=True,
                        input=raster,config=configfile,output=filename)
    outputfile=os.path.join(os.path.split(configfile)[0],"output",filename)
    return outputfile
    
def get_classlevel_metrics(configfile, raster, returnlistresult=True, ncores=2):
    # Check if raster exists to avoid error in mutliprocessing 
    try:
        mpset=raster.split("@")[1]
    except:
        mpset=""
    if raster not in [x.split("@")[0] for x in gscript.list_strings(type='raster',mapset=mpset)]:
        sys.exit('Raster <%s> not found' %raster)
    # Check if configfile exists to avoid error in mutliprocessing 
    if not os.path.exists(configfile):
        sys.exit('Configuration file <%s> not found' %configfile)
    # List of metrics to be computed
    spatial_metric_list=["patchnum","patchdensity","mps","padcv","padrange","padsd","shape","edgedensity"]
    # Check for number of cores doesnt exceed available
    nbcpu=multiprocessing.cpu_count()
    if ncores>=nbcpu:
        ncores=nbcpu-1
        if ncores>len(spatial_metric_list):
            ncores=len(spatial_metric_list)  #Adapt number of cores to number of metrics to compute
    # Declare empty list for return
    returnlist=[] 
    # Create a new pool
    p=Pool(ncores)
    # Set the two fixed argument of the 'compute_classlevel_metrics' function
    func=partial(compute_classlevel_metrics,configfile, raster)
    # Launch the processes for as many items in the 'functions_name' list and get the ordered results using map function
    returnlist=p.map(func,spatial_metric_list)
    p.close()
    p.join()
    # Return list of paths to result files
    if returnlistresult:
        return returnlist

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

<center> <font size=5> <h1>User inputs</h1> </font> </center> 

In [195]:
## Define a empty dictionnary for saving user inputs
user={}

In [196]:
## Enter the path to GRASSDATA folder
user["gisdb"] = "/home/tais/Documents/GRASSDATA_Spie2017subset_Ouaga"

## Enter the name of the location (existing or for a new one)
user["location"] = "SPIE_subset"

## Enter the EPSG code for this location 
user["locationepsg"] = "32630"

## Enter the name of the mapset to use for segmentation
user["mapsetname"] = "test_rli"

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Compute spatial metrics for deriving land use in street blocs

**Launch GRASS GIS working session**

In [197]:
## Set the name of the mapset in which to work
mapsetname=user["mapsetname"]

## Launch GRASS GIS working session in the mapset
if os.path.exists(os.path.join(user["gisdb"],user["location"],mapsetname)):
    gsetup.init(os.environ['GISBASE'], user["gisdb"], user["location"], mapsetname)
    print "You are now working in mapset '"+mapsetname+"'" 
else: 
    print "'"+mapsetname+"' mapset doesn't exists in "+user["gisdb"]

You are now working in mapset 'test_rli'


**Set the path to the r.li folder for configuration file and for results**

In [198]:
os.environ

{'MDMSESSION': 'mate', 'MANDATORY_PATH': '/usr/share/gconf/mate.mandatory.path', 'MATE_DESKTOP_SESSION_ID': 'this-is-deprecated', 'LESSOPEN': '| /usr/bin/lesspipe %s', 'MDM_LANG': 'fr_BE.UTF-8', 'LOGNAME': 'tais', 'USER': 'tais', 'HOME': '/home/tais', 'XDG_VTNR': '9', 'PATH': '/usr/local/bin:/home/tais/BIN:/home/tais/bin:/home/tais/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/bin:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/script:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/lib:/home/tais/.grass7/addons:/home/tais/.grass7/addons/bin:/home/tais/.grass7/addons:/home/tais/.grass7/addons/scripts:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/bin:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/scripts:/home/tais/.grass7/addons/scripts:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/bin:/home/tais/SRC/GRASS/grass_trunk/dist

In [200]:
# Define path of the outputfile (in r.li folder)
version=grass.version()['version'].split('.')[0] # Get the version of GRASS GIS 
if sys.platform=="win32":
    rli_config_dir=os.path.join(os.environ['APPDATA'],"GRASS"+version,"r.li")
    rli_output_dir=os.path.join(os.environ['APPDATA'],"GRASS"+version,"r.li","output")
else: 
    rli_config_dir=os.path.join(os.environ['HOME'],".grass"+version,"r.li")
    rli_output_dir=os.path.join(os.environ['HOME'],".grass"+version,"r.li","output")
if not os.path.exists(rli_config_dir):
    os.makedirs(rli_config_dir)
if not os.path.exists(rli_output_dir):
    os.makedirs(rli_output_dir)
# Print
print "GRASS GIS add-on's r.li configuration files will be saved under <%s>."%(rli_config_dir,)
print "GRASS GIS add-on's r.li outputs will be saved under <%s>."%(rli_output_dir,)

GRASS GIS add-on's r.li configuration files will be saved under </home/tais/.grass7/r.li>.
GRASS GIS add-on's r.li outputs will be saved under </home/tais/.grass7/r.li/output>.


**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

### Define the name of the base landcover map and landscape units polygons

In [201]:
# Set the name of the 'base' landcover map
baselandcoverraster="classif@test_rli"
# Set the name of the vector polygon layer containing the landscape units
landscape_polygons="streetblocks"

### Import shapefile containing street blocks polygons

In [202]:
# Set the path to the shapefile containing streetblocks polygons
pathtoshp="/media/tais/data/Dropbox/ULB/MAUPP/Landuse_mapping/Test_spatial_metrics_computation/Data/streetblocks_subset.shp"

In [203]:
# Import shapefile
gscript.run_command('v.in.ogr', quiet=True, overwrite=True, input=pathtoshp, output=landscape_polygons)

0

### Create binary rasters from the base landcover map

In [204]:
# Save time for computing processin time
begintime=time.time()

# Create as many binary raster layer as categorical values existing in the base landcover map
gscript.run_command('g.region', raster=baselandcoverraster, quiet=True) #Set the region
pref=baselandcoverraster.split("@")[0]+"_cl"  #Set the prefix

raster_list=[]  # Initialize a empty list for results
raster_list=create_binary_raster(baselandcoverraster,
                                 prefix=pref,setnull=True,returnlistraster=True,
                                 category_list=None,ncores=15)  #Extract binary raster 

# Compute and print processing time
print_processing_time(begintime,"Extraction of binary rasters achieved in ")

'Extraction of binary rasters achieved in 4.3 seconds'

In [205]:
# Insert the name of the base landcover map at first position in the list
raster_list.insert(0,baselandcoverraster)
# Display the raster to be used for landscape analysis
raster_list

['classif@test_rli',
 'classif_cl_11',
 'classif_cl_13',
 'classif_cl_14',
 'classif_cl_20',
 'classif_cl_30',
 'classif_cl_31',
 'classif_cl_41',
 'classif_cl_51']

## Create r.li configuration file for a list of landcover rasters

In [206]:
# Save time for computing processin time
begintime=time.time()
# Run creation of r.li configuration file and associated raster layers
list_cats,list_configfile=create_rli_configfile(raster_list,landscape_polygons,masklayerhardcopy=False,returnlistpath=True,ncores=20)
# Compute and print processing time
print_processing_time(begintime,"Creation of r.li configuration files achieved in ")

'Creation of r.li configuration files achieved in 3 minutes and 32.1 seconds'

In [207]:
# Display the path to the configuration files created
list_configfile

[u'/home/tais/.grass7/r.li/classif_streetblocks',
 u'/home/tais/.grass7/r.li/classif_cl_11_streetblocks',
 u'/home/tais/.grass7/r.li/classif_cl_13_streetblocks',
 u'/home/tais/.grass7/r.li/classif_cl_14_streetblocks',
 u'/home/tais/.grass7/r.li/classif_cl_20_streetblocks',
 u'/home/tais/.grass7/r.li/classif_cl_30_streetblocks',
 u'/home/tais/.grass7/r.li/classif_cl_31_streetblocks',
 u'/home/tais/.grass7/r.li/classif_cl_41_streetblocks',
 u'/home/tais/.grass7/r.li/classif_cl_51_streetblocks']

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

## Compute spatial metrics at landscape level

In [208]:
# Initialize an empty list which will contains the resultfiles 
resultfiles=[]

In [209]:
# Save time for computing processin time
begintime=time.time()
# Get the path to the configuration file for the base landcover raster
currentconfigfile=list_configfile[0]
# Get the name of the base landcover raster
currentraster=raster_list[0]
# Set the region to match the extent of the base raster
gscript.run_command('g.region', raster=currentraster, quiet=True)
# Launch the processes for as many items in the 'functions_name' list and get the ordered results using map function
resultfiles.append(get_landscapelevel_metrics(currentconfigfile, currentraster, returnlistresult=True, ncores=15))
# Compute and print processing time
print_processing_time(begintime,"Computation of spatial metric achieved in ")

'Computation of spatial metric achieved in 42.6 seconds'

In [210]:
resultfiles

[[u'/home/tais/.grass7/r.li/output/classif_dominance',
  u'/home/tais/.grass7/r.li/output/classif_pielou',
  u'/home/tais/.grass7/r.li/output/classif_renyi',
  u'/home/tais/.grass7/r.li/output/classif_richness',
  u'/home/tais/.grass7/r.li/output/classif_shannon',
  u'/home/tais/.grass7/r.li/output/classif_simpson']]

## Compute spatial metrics at class level

In [211]:
# Save time for computing processin time
begintime=time.time()
# Get a list with paths to the configuration file for class level metrics
classlevelconfigfiles=list_configfile[1:]
# Get a list with name of binary landcover raster for class level metrics
classlevelrasters=raster_list[1:]

for x,currentraster in enumerate(classlevelrasters[:]):
    # Get the path to the configuration file for the base landcover raster
    currentconfigfile=classlevelconfigfiles[x]
    # Set the region to match the extent of the base raster
    gscript.run_command('g.region', raster=currentraster, quiet=True)
    # Launch the processes for as many items in the 'functions_name' list and get the ordered results using map function
    resultfiles.append(get_classlevel_metrics(currentconfigfile, currentraster, returnlistresult=True, ncores=10))

# Compute and print processing time
print_processing_time(begintime,"Computation of spatial metric achieved in ")

'Computation of spatial metric achieved in 5 minutes and 57.6 seconds'

In [212]:
resultfiles

[[u'/home/tais/.grass7/r.li/output/classif_dominance',
  u'/home/tais/.grass7/r.li/output/classif_pielou',
  u'/home/tais/.grass7/r.li/output/classif_renyi',
  u'/home/tais/.grass7/r.li/output/classif_richness',
  u'/home/tais/.grass7/r.li/output/classif_shannon',
  u'/home/tais/.grass7/r.li/output/classif_simpson'],
 [u'/home/tais/.grass7/r.li/output/classif_cl_11_patchnum',
  u'/home/tais/.grass7/r.li/output/classif_cl_11_patchdensity',
  u'/home/tais/.grass7/r.li/output/classif_cl_11_mps',
  u'/home/tais/.grass7/r.li/output/classif_cl_11_padcv',
  u'/home/tais/.grass7/r.li/output/classif_cl_11_padrange',
  u'/home/tais/.grass7/r.li/output/classif_cl_11_padsd',
  u'/home/tais/.grass7/r.li/output/classif_cl_11_shape',
  u'/home/tais/.grass7/r.li/output/classif_cl_11_edgedensity'],
 [u'/home/tais/.grass7/r.li/output/classif_cl_13_patchnum',
  u'/home/tais/.grass7/r.li/output/classif_cl_13_patchdensity',
  u'/home/tais/.grass7/r.li/output/classif_cl_13_mps',
  u'/home/tais/.grass7/r.li/

### Change the results files from r.li to get the correct 'cat' value for each landscape unit

In [215]:
import csv, shutil
from itertools import izip

for f in resultfiles:
    f_in=open(f)
    f_tmp=open(f+'_tmp',"w")
    f_in_reader=csv.reader(f_in,delimiter='|')
    f_tmp_writer=csv.writer(f_tmp,delimiter='|')
    f_tmp_writer.writerow(['cat',"_".join(os.path.split(f)[-1].split("_")[1:])])
    for i,row in enumerate(f_in_reader):
        newrow=[]
        try:
            newrow.append(list_cats[i])
            newrow.append(row[1])
            f_tmp_writer.writerow(newrow)
        except:
            continue
    f_in.close()
    f_tmp.close()
    os.remove(f)
    shutil.copy2(f+'_tmp',f)
    os.remove(f+'_tmp')

# Compute some special metrics

### Creating a raster layer of landscape units

Creating a raster layer of landscape units with resolution corresponding to the one of land cover raster

In [216]:
# Create a raster corresponding to the landscape units (for computing statistics using i.segment.stats)
gscript.run_command('g.region', raster=baselandcoverraster, quiet=True) #Set the region
raster_landscapeunits="temp_%s"%landscape_polygons.split("@")[0]
gscript.run_command('v.to.rast', overwrite=True, input=landscape_polygons, output=raster_landscapeunits, use='cat') 

0

### Shape statistics for the landscape units

In [169]:
# Set the path to the file for i.segment.stats results for landscape units shape metrics
landscape_units_shape_metrics=os.path.join(rli_output_dir,"landscape_units_shape_metrics")

In [170]:
# Save time for computing processin time
begintime=time.time()
# Run i.segment.stats
gscript.run_command('i.segment.stats', overwrite=True, map=raster_landscapeunits,
                    area_measures='area,perimeter,compact_circle,compact_square,fd',
                    csvfile=landscape_units_shape_metrics,
                    processes='1')

# Compute and print processing time
print_processing_time(begintime,"Metrics computed in ")

'Metrics computed in 1.3 seconds'

In [171]:
resultfiles.append(landscape_units_shape_metrics)

### Mean and standard deviation of SAR textures, NDVI, NDWI

In [172]:
# Set the name of the nDSM layer
ndsm="ndsm"
# Set the name of the NDVI layer
ndvi="ndvi"
# Set the name of the NDWI layer
ndwi="ndwi"
# Set the prefix of SAR textures layer
SAR_prefix="SAR_w"

In [173]:
# Set up a list with name of raster layer to be used
ancillarylayers=[]
ancillarylayers.append(ndvi)
ancillarylayers.append(ndwi)
[ancillarylayers.append(x) for x in gscript.list_strings("rast", pattern=SAR_prefix, flag='r')] #Append SAR textures
print "Layer to be used :\n\n"+'\n'.join(ancillarylayers)

Layer to be used :

ndvi
ndwi
SAR_w11.1@PERMANENT
SAR_w11.2@PERMANENT
SAR_w11.3@PERMANENT
SAR_w11.4@PERMANENT
SAR_w11.5@PERMANENT
SAR_w11.6@PERMANENT
SAR_w11.7@PERMANENT
SAR_w7.1@PERMANENT
SAR_w7.2@PERMANENT
SAR_w7.3@PERMANENT
SAR_w7.4@PERMANENT
SAR_w7.5@PERMANENT
SAR_w7.6@PERMANENT
SAR_w7.7@PERMANENT


In [174]:
# Set the path to the file for i.segment.stats results for metrics_ndvi_ndwi_sar
metrics_ndvi_ndwi_sar=os.path.join(rli_output_dir,"metrics_ndvi_ndwi_sar")

In [175]:
# Save time for computing processin time
begintime=time.time()
###### Compute shape metrics as well as mean and stddev of ancillary layers for each landscape unit
## Set number of cores to be used
ncores=len(ancillarylayers) 
nbcpu=multiprocessing.cpu_count()
if ncores>=nbcpu:
    ncores=nbcpu-1
    if ncores>len(ancillarylayers):
        ncores=len(ancillarylayers)  #Adapt number of cores to number of metrics to compute
# Run i.segment.stats
gscript.run_command('i.segment.stats', overwrite=True, map=raster_landscapeunits,
                    raster_statistics='stddev,median',
                    rasters=','.join(ancillarylayers),
                    csvfile=metrics_ndvi_ndwi_sar,
                    processes=ncores,
                    flags='s')

# Compute and print processing time
print_processing_time(begintime,"Metrics computed in ")

'Metrics computed in 10.9 seconds'

In [176]:
resultfiles.append(metrics_ndvi_ndwi_sar)

In [177]:
resultfiles

[u'/home/tais/.grass7/r.li/output/classif_dominance',
 u'/home/tais/.grass7/r.li/output/classif_pielou',
 u'/home/tais/.grass7/r.li/output/classif_renyi',
 u'/home/tais/.grass7/r.li/output/classif_richness',
 u'/home/tais/.grass7/r.li/output/classif_shannon',
 u'/home/tais/.grass7/r.li/output/classif_simpson',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_patchnum',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_patchdensity',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_mps',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_padcv',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_padrange',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_padsd',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_shape',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_edgedensity',
 u'/home/tais/.grass7/r.li/output/classif_cl_13_patchnum',
 u'/home/tais/.grass7/r.li/output/classif_cl_13_patchdensity',
 u'/home/tais/.grass7/r.li/output/classif_cl_13_mps',
 u'/home/tais/.grass7/r.li/output/classif_cl_13

### Mean and standard deviation of building's height

#### Create raster with nDSM value of 'buildings' pixels

In [178]:
# Set pixel value of 'buildings' on the 'baselandcoverraster'
buildpixel=11
# Set the name of the new layer containing height of buildings
buildings_height='buildings_height'
# Set the path to the file for i.segment.stats results for metrics_ndvi_ndwi_sar
metrics_buildings_height=os.path.join(rli_output_dir,"metrics_buildings_height")
# Create temp fil which will contain intermediate results
TMP_sumheights=grass.tempfile()+'_sumheights.csv'
TMP_nbrbuildpixels=grass.tempfile()+'_nbrbuildpixels.csv'

In [179]:
# Save time for computing processin time
begintime=time.time()
# Create a raster layer with height of pixels classified as 'buildings'
gscript.run_command('g.region', raster=baselandcoverraster, quiet=True) #Set the region
formula="%s=if(%s==%s, %s, 0)"%(buildings_height,baselandcoverraster,buildpixel,ndsm)
gscript.mapcalc(formula, overwrite=True)
# Compute and print processing time
print_processing_time(begintime,"Creation of layer in ")

'Creation of layer in 5.9 seconds'

In [180]:
# Save time for computing processin time
begintime=time.time()
# Compute sum of build pixels's height using i.segment.stats 
gscript.run_command('i.segment.stats', overwrite=True, map=raster_landscapeunits,
                    raster_statistics='sum', flags='s', rasters='buildings_height',
                    csvfile=TMP_sumheights,processes=ncores)
# Compute number of built pixels using i.segment.stats
binary_builtup_raster="%s_cl_%s"%(baselandcoverraster.split("@")[0],buildpixel)
gscript.run_command('g.copy', overwrite=True, raster='%s,tmp'%binary_builtup_raster)
gscript.run_command('r.null', map='tmp', null=0)
gscript.run_command('i.segment.stats', overwrite=True, map=raster_landscapeunits,
                    raster_statistics='sum', flags='s', rasters='tmp',
                    csvfile=TMP_nbrbuildpixels,processes=ncores)
# Compute and print processing time
print_processing_time(begintime,"i.segment.stats run in ")

'i.segment.stats run in 10.8 seconds'

In [181]:
# Save time for computing processin time
begintime=time.time()
# Improt library to be able to iterate on two files in the same loop
from itertools import izip
# Declare empty dictionnary
tmp_dic={}
for i, (line_from_file_1, line_from_file_2) in enumerate(izip(open(TMP_sumheights), open(TMP_nbrbuildpixels))):
    if i==0:
        continue
    f1_items=line_from_file_1.split("\n")[0].split("|")
    f2_items=line_from_file_2.split("\n")[0].split("|")
    key=f1_items[0]
    sumheight=f1_items[1]
    nbpixel=f2_items[1]
    try:
        mean_height=float(sumheight)/float(nbpixel)
    except ZeroDivisionError:
        mean_height=0
    tmp_dic[key]=mean_height
# Get the name of the first colum
with open(TMP_sumheights) as f:
    column_a=f.next().split("\n")[0].split("|")[0]
# Built the content of the file
content=[]
content.append((column_a,'mean_build_height'))
for key in tmp_dic.keys():
    content.append((key,tmp_dic[key]))
# Create a new file
fout=open(metrics_buildings_height,"w")
writer=csv.writer(fout, delimiter='|')
writer.writerows(content)
fout.close()
# Compute and print processing time
print_processing_time(begintime,"Mean build pixels's height computed in ")

"Mean build pixels's height computed in 0.0 seconds"

In [182]:
# Remove temporary layers
gscript.run_command('g.remove', flags='ef', type='raster', name='tmp')
# Remove temporary files
os.remove(TMP_sumheights)
os.remove(TMP_nbrbuildpixels)

In [183]:
resultfiles.append(metrics_buildings_height)

In [295]:
resultfiles

[u'/home/tais/.grass7/r.li/output/classif_dominance',
 u'/home/tais/.grass7/r.li/output/classif_pielou',
 u'/home/tais/.grass7/r.li/output/classif_renyi',
 u'/home/tais/.grass7/r.li/output/classif_richness',
 u'/home/tais/.grass7/r.li/output/classif_shannon',
 u'/home/tais/.grass7/r.li/output/classif_simpson',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_patchnum',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_patchdensity',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_mps',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_padcv',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_padrange',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_padsd',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_shape',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_edgedensity',
 u'/home/tais/.grass7/r.li/output/classif_cl_13_patchnum',
 u'/home/tais/.grass7/r.li/output/classif_cl_13_patchdensity',
 u'/home/tais/.grass7/r.li/output/classif_cl_13_mps',
 u'/home/tais/.grass7/r.li/output/classif_cl_13

### Proportion of each of individual classes in the landcover map

In [292]:
### Function that compute the proportion of each class of landcover
import itertools
import multiprocessing
from functools import partial
import sys, csv
import grass.script as gscript

def random_string(N):
    import random, string
    prefix=random.choice(string.ascii_uppercase + string.ascii_lowercase)
    suffix=''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(N))
    return prefix+suffix

def compute_prop_landcover(outputfolder,landscape_units_raster,binary_class_raster):
    # Save the current class in a new variable 
    current_class=binary_class_raster.split("@")[0].split("_")[-1]
    # Initialize a temp files
    temp_file=os.path.join(outputfolder,random_string(10))
    tmp_copy='%s_%s'%(random_string(4),current_class)
    # Make copy of the raster and fill the null values with zero
    gscript.run_command('g.copy', overwrite=True, raster='%s,%s'%(binary_class_raster,tmp_copy))
    gscript.run_command('r.null', map=tmp_copy, null='0')
    # Compute area of landscape unit and total pixels of the current binary class raster
    gscript.run_command('i.segment.stats', flags="r", quiet=True, overwrite=True, 
                        map=landscape_units_raster, area_measures='area', 
                        rasters=tmp_copy, raster_statistics='sum', 
                        csvfile=temp_file, separator='pipe')
    gscript.run_command('g.remove', flags='f', type='raster', name=tmp_copy)
    # Create outputfile and compute proportion of class
    outfile=os.path.join(outputfolder, "prop_%s"%current_class)
    writer=csv.writer(open(outfile,'w'), delimiter="|")
    reader=csv.reader(open(temp_file,'r'), delimiter="|")
    header=reader.next()
    crash_report=[]
    content=[]
    content.append([header[0],"prop_%s"%current_class])  #Create new header with first original column and current class related name for proportion
    for row in reader:
        try:
            prop=100*float(row[2])/float(row[1])
            content.append([row[0],"{0:.5f}".format(prop)])
        except ZeroDivisionError:
            crash_report.append(row[0])
            continue
    writer.writerows(content)
    os.remove(temp_file)
    # Print notification of ZeroDivisionError if it happened
    if len(crash_report)>0:
        print "An 'ZeroDivisionError' has been registered for the following <%s>"%header[0]+"\n".join(crash_report)
    # Return the result file
    return outfile
    
def get_classproportions(outputfolder,landscape_units_raster,binary_class_raster_list,
                         returnlistresult=True,ncores=2):
    # Check if raster exists to avoid error in mutliprocessing 
    try:
        mpset=landscape_units_raster.split("@")[1]
    except:
        mpset=""
    if landscape_units_raster not in [x.split("@")[0] for x in gscript.list_strings(type='raster',mapset=mpset)]:
        sys.exit('Raster <%s> not found' %landscape_units_raster)
    # Check for number of cores doesnt exceed available
    nbcpu=multiprocessing.cpu_count()
    if ncores>=nbcpu:
        ncores=nbcpu-1
        if ncores>len(binary_class_raster_list):
            ncores=len(binary_class_raster_list)  #Adapt number of cores to number of metrics to compute
    # Declare empty list for return
    returnlist=[] 
    # Create a new pool
    p=Pool(ncores)
    # Set the two fixed argument of the 'compute_prop_landcover' function
    func=partial(compute_prop_landcover,outputfolder,landscape_units_raster)
    # Launch the processes for as many items in the 'functions_name' list and get the ordered results using map function
    returnlist=p.map(func,binary_class_raster_list)
    p.close()
    p.join()
    # Return list of paths to result files
    if returnlistresult:
        return returnlist

In [293]:
# Set a tuple with name of land cover class binary raster to be processed 
listofbinary=tuple(raster_list[1:])
# Compute proportion of each land cover class in landscape units
proportion_results=get_classproportions(rli_output_dir,raster_landscapeunits,listofbinary,returnlistresult=True,ncores=10)

In [296]:
resultfiles.append(proportion_results)

[None, None, None, None, None, None, None, None]

In [297]:
resultfiles

[u'/home/tais/.grass7/r.li/output/classif_dominance',
 u'/home/tais/.grass7/r.li/output/classif_pielou',
 u'/home/tais/.grass7/r.li/output/classif_renyi',
 u'/home/tais/.grass7/r.li/output/classif_richness',
 u'/home/tais/.grass7/r.li/output/classif_shannon',
 u'/home/tais/.grass7/r.li/output/classif_simpson',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_patchnum',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_patchdensity',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_mps',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_padcv',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_padrange',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_padsd',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_shape',
 u'/home/tais/.grass7/r.li/output/classif_cl_11_edgedensity',
 u'/home/tais/.grass7/r.li/output/classif_cl_13_patchnum',
 u'/home/tais/.grass7/r.li/output/classif_cl_13_patchdensity',
 u'/home/tais/.grass7/r.li/output/classif_cl_13_mps',
 u'/home/tais/.grass7/r.li/output/classif_cl_13

In [213]:
# Flat the 'resultfiles' list which contains several lists
resultfiles=[item for sublist in resultfiles for item in sublist]

# Combine all .csv files together

In [298]:
def atoi(text):
    '''
    Function that return integer if text is digit - Used in 'natural_keys' function
    '''
    return int(text) if text.isdigit() else text

def natural_keys(text):   #     Trick was found here: https://stackoverflow.com/questions/5967500/how-to-correctly-sort-a-string-with-a-number-inside
    '''
    Return key to be used for sorting string containing numerical values - Used in 'leftjoin_2csv' function
    '''
    import re  #Import needed library
    return [ atoi(c) for c in re.split('(\d+)', text) ]  #Split the string

def join_2csv(file1,file2,separator=";",join='inner',fillempty='NULL'):
    '''
    Function that join two csv files according to the first column (primary key).
    'file1' and 'file2' wait for complete path (strings) to the corresponding files. Please not that 'file1' is assume to be the left-one in the join
	'separator' wait for the character to be considered as .csv delimiter (string)
	'join' parameter wait either for 'left' or 'inner' according to type of join
	'fillempty' wait for the string to be use to fill the blank when no occurance is found for the join operation
    '''
    import tempfile,csv,os
    header_list=[]
    file1_values_dict={}
    file2_values_dict={}
    reader1=csv.reader(open(file1), delimiter=separator) #Csv reader for file 1
    reader2=csv.reader(open(file2), delimiter=separator) #Csv reader for file 2
    # Make a list of headers
    header_list1=[ x for x in reader1.next()]
    header_list2=[ x for x in reader2.next()[1:]]
    # Make a list of unique IDs from the first and second table according to type of join
    if join=='inner':
        id_list=[ row[0] for row in reader1]
        [id_list.append(row[0]) for row in reader2]
        id_list=list(set(id_list))
        id_list.sort(key=natural_keys)
    if join=='left':
        id_list=[ row[0] for row in reader1]
        id_list=list(set(id_list))
        id_list.sort(key=natural_keys)
    # Build dictionnary for values of file 1
    reader1=csv.reader(open(file1), delimiter=separator)
    reader1.next()
    values_dict1={rows[0]:rows[1:] for rows in reader1}
    # Build dictionnary for values of file 2
    reader2=csv.reader(open(file2), delimiter=separator)
    reader2.next()
    values_dict2={rows[0]:rows[1:] for rows in reader2}
    # Built new content
    new_content=[]
    new_header=header_list1+header_list2
    new_content.append(new_header)
    for key in id_list:
        new_row=[key]
        try:
            [new_row.append(value) for value in values_dict1[key]]
        except:
            [new_row.append('%s'%fillempty) for x in header_list1[1:]]
        try:
            [new_row.append(value) for value in values_dict2[key]]
        except:
            [new_row.append('%s'%fillempty) for x in header_list2]
        new_content.append(new_row)
    #Return the result
    outfile=os.path.join(tempfile.gettempdir(),"temp")
    writer=csv.writer(open(outfile,"w"), delimiter=separator)
    writer.writerows(new_content) #Write multiples rows in the file
    return outfile

def join_multiplecsv(fileList,outfile,separator=";",join='inner', fillempty='NULL', overwrite=False):
    '''
    Function that apply join on multiple csv files
    '''
    import os, sys, shutil
    # Stop execution if outputfile exitst and can not be overwriten
    if os.path.isfile(outfile) and overwrite==False:
        print "File '%s' aleady exists and overwrite option is not enabled."%outfile
    else:
        if os.path.isfile(outfile) and overwrite==True:  # If outputfile exitst and can be overwriten
            #os.remove(outfile)
            print "File '%s' will be overwrited."%outfile
        nbfile=len(fileList)
        if nbfile<=1: #Check if there are at least 2 files in the list
            sys.exit("This function require at least two .csv files to be jointed together.")
        # Copy the list of file in a queue list
        queue_list=list(fileList)
        # Left join on the two first files
        file1=queue_list.pop(0)
        file2=queue_list.pop(0)
        tmp_file=join_2csv(file1,file2,separator=separator,join=join, fillempty=fillempty)
        # Left join on the rest of the files in the list
        while len(queue_list)>0:
            file2=queue_list.pop(0)
            tmp_file=join_2csv(tmp_file,file2,separator=separator,join=join, fillempty=fillempty)
        #Copy the temporary file to the desired output path
        shutil.copy2(tmp_file,outfile)
        # Print what happend
        print "%s individual .csv files were joint together."%nbfile

def create_csvt(csv_file,separator=";",first_col_type="Integer",rest_type="Real"):
    '''
    Function that create a .csvt file with the same type of all columns except first one
    '''
    import csv
    writer=csv.writer(open(csv_file+"t","w"),delimiter=separator)
    reader=csv.reader(open(csv_file,"r"),delimiter=separator)
    header=reader.next()
    typecolumn=[]
    typecolumn.append(first_col_type)
    for columns in header[1:]:
        typecolumn.append(rest_type)
    writer.writerow(typecolumn)

In [301]:
# Define the path for the .csv with final results
outfile=os.path.join(rli_output_dir,"land_use_metrics.csv")

In [302]:
# Join (inner join) all result files together in a new .csv file
join_multiplecsv(resultfiles,outfile,separator="|",join='inner',overwrite=True)
# Create .csvt file
create_csvt(outfile,separator="|",first_col_type="Integer",rest_type="Real")

File '/home/tais/.grass7/r.li/output/land_use_metrics.csv' will be overwrited.
78 individual .csv files were joint together.


### Make a copy of the .csv file with results, where 'null' values are empty cells 

**For .csvt**

In [ ]:
# Define path to the .csv files
afile=outfile+'t'
pathtofile,extension=os.path.splitext(afile)
bfile=pathtofile+"_blanknull"+extension
# Make copy of the file
shutil.copy2(afile,bfile)

**For .csv**

In [ ]:
# Define path to the .csv files
afile=outfile
pathtofile,extension=os.path.splitext(afile)
outfile_withoutnull=pathtofile+"_blanknull"+extension

In [ ]:
import csv
# Create a copy by removing the 'NULL' values
reader=csv.reader(open(afile,'r'),delimiter="|")
writer=csv.writer(open(outfile_withoutnull,'w'),delimiter="|")
for row in reader:
    newline=[]
    [newline.append(x) if x != "NULL" else newline.append("") for x in row]
    writer.writerow(newline)

### Check if all the lines have the same number of items

In [ ]:
#### Function that check if rows of a .csv file have the same number of items than number of column in the first line (header)
### If rows's lenght are identical, the function return 0. If not, it return a list containing the index number of row(s) that failed
### The index is starting with value 1 for the first line of data (the header row is indexed 0)
# 'csvfile' wait for the complete path (string) to the csvfile to be checked
# 'separator' wait for the character used as separator in the .csv file (string)
# 'allowemptycell' wait for a boolean value (True, False) depending if a empty cell should not be considered as a problem in the file

def check_lenght_row(csvfile,separator,allowemptycell=True):
    listofunequalrow=[]
    afile=open(csvfile,'r')
    header=afile.next()  # Save header row and go to the next one
    nb_item_control=len(header.split(separator))
    for x,row in enumerate(afile,1):    # Start counting at "1" because header is already skipped
        row_items=row.split(separator)
        if len(row_items) != nb_item_control: # Check it number of items is identical than the number of items in the header
            listofunequalrow.append(x)
        if not allowemptycell:
            if "" in row_items:
                listofunequalrow.append(x)
    listofunequalrow=list(set(listofunequalrow))   # Recreate a list with uniques values from the original list
    if len(listofunequalrow)>0:
        return listofunequalrow  # Return a list of indexes for line whose length is not equal to the header
    else:
        return 0

In [ ]:
# Check number of row in the .csv file
fout=check_lenght_row(outfile_withoutnull,";",allowemptycell=True)
if fout>0:
    print "Rows in csv do not have the same lenght. Please check the following row(s) index(es)."
    print "/n".join(fout)
else:
    print "The csv looks good."

### Replace current delimiter by ';'

In [ ]:
import csv
reader=csv.reader(open(outfile,'r'),delimiter="|")
newfile=[]
for row in reader:
    newline=[]
    [newline.append(x) for x in row]
    newfile.append(newline)
writer=csv.writer(open(outfile,'w'),delimiter=";")
writer.writerows(newfile)

In [ ]:
import csv
reader=csv.reader(open(outfile_withoutnull,'r'),delimiter="|")
newfile=[]
for row in reader:
    newline=[]
    [newline.append(x) for x in row]
    newfile.append(newline)
writer=csv.writer(open(outfile_withoutnull,'w'),delimiter=";")
writer.writerows(newfile)

### Display the .csv using pandas

In [ ]:
import pandas as pd
# Load the .csv file in a pandas dataframe
df=pd.read_csv(outfile_withoutnull, sep=';',header=0)
# Display the dataframe
df

### Move files to dedicated folder

**Configuration files**

In [ ]:
# Set the folder where to move the configuration files
finalfolder='/home/tais/Documents/GRASSDATA_Spie2017subset_Ouaga/Results_spatial_metrics/rli_config'

In [ ]:
## Create the folder if does not exists
if not os.path.exists(finalfolder):
    os.makedirs(finalfolder)
    print "Folder '"+finalfolder+"' created"
## Copy the files to the final folder and remove them from the original folder
for configfile in list_configfile:
    shutil.copy2(configfile,finalfolder)
    os.remove(configfile)

**Result files**

In [ ]:
# Set the folder where to move the configuration files
finalfolder='/home/tais/Documents/GRASSDATA_Spie2017subset_Ouaga/Results_spatial_metrics/rli_results'

In [ ]:
## Create the folder if does not exists
if not os.path.exists(finalfolder):
    os.makedirs(finalfolder)
    print "Folder '"+finalfolder+"' created"
## Copy the files to the final folder and remove them from the original folder
for res_file in resultfiles:
    shutil.copy2(res_file,finalfolder)
    os.remove(res_file)

In [ ]:
# Copy the final csv file with all the results
shutil.copy2(outfile,finalfolder)
os.remove(outfile)
shutil.copy2(outfile+'t',finalfolder)
os.remove(outfile+'t')

In [ ]:
# Reasign the new path to the variable
outfile=os.path.join(finalfolder,os.path.split(outfile)[-1])

In [ ]:
# Copy the final csv file with all the results
shutil.copy2(outfile_withoutnull,finalfolder)
os.remove(outfile_withoutnull)
shutil.copy2(outfile_withoutnull+'t',finalfolder)
os.remove(outfile_withoutnull+'t')

In [ ]:
# Reasign the new path to the variable
outfile_withoutnull=os.path.join(finalfolder,os.path.split(outfile_withoutnull)[-1])

## Export the landscape polygons (with 'cat' column) as shapefile

In [ ]:
# Define the name of the shapefile with landscape units and the 'cat' column
outputshp='/home/tais/Documents/GRASSDATA_Spie2017subset_Ouaga/Results_spatial_metrics/shapefile/Ouaga_subset_streetblocks.shp'

In [ ]:
## Create the folder if does not exists
dirname=os.path.dirname(outputshp)
if not os.path.exists(dirname):
    os.makedirs(dirname)
    print "Folder '"+dirname+"' created"

In [ ]:
# Export vector layer as shapefile
gscript.run_command('v.out.ogr', flags='cem', overwrite=True, 
                    input=landscape_polygons, output=outputshp, format='ESRI_Shapefile')

## Join the .csv file to the landscape unit polygon layer

In [ ]:
# Import .csv as new table in GRASS
csvfile=outfile
gscript.run_command('db.in.ogr', overwrite=True, quiet=True, input=csvfile, output='spatial_metrics_table')

In [ ]:
# Join the vector layer with the new table
gscript.run_command('v.db.join', quiet=True, map=landscape_polygons, column='cat', 
                    other_table='spatial_metrics_table', other_column='cat_')

In [185]:
remove temp_streetblocks raster layer

SyntaxError: invalid syntax (<ipython-input-185-ebf05269556d>, line 1)

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**